# This is how you compute a derivative

In [1]:
import theano
from theano import tensor as T

In [9]:
x = T.dscalar('x')
y = x**2 + 2

fn = theano.function([x], y)

In [15]:
dy_by_dx = T.grad(y, x)
deriv = theano.function([x], dy_by_dx)

# This is how you use a shared variable

In [17]:
from theano import shared

In [25]:
state = shared(0)
inc = T.iscalar('inc')
increment = theano.function([inc], state, updates = [(state, state + inc)])

In [26]:
print state.get_value()
increment(5)
print state.get_value()

0
5


# Scan

In [2]:
k = T.iscalar("k")
A = T.iscalar("A")

In [3]:
results, updates = theano.scan(fn = lambda prior_result: prior_result * A, 
                                                         outputs_info = A,
                                                         n_steps = k)

In [52]:
final_result = results[-1]

OrderedUpdates()

In [54]:
power = theano.function(inputs = [A, k], outputs = results, updates = updates)

In [55]:
power(2, 4)

array([ 4,  8, 16, 32], dtype=int32)

In [29]:
i = T.iscalar('i')
results, updates = theano.scan(fn=lambda previous_count: previous_count * 1,
                            outputs_info=0,
                            n_steps=i)

# Scan ---> Factorial  (doesn't work)

In [121]:
results, updates = theano.scan(fn = lambda previous: previous * (A - 1),
                                                     outputs_info = A,
                                                     n_steps = A)

In [122]:
factorial = theano.function([A], outputs = results, updates = updates)

In [123]:
factorial(4)

array([ 12,  36, 108, 324], dtype=int32)

###Peter's Thoughts

I would change a number of things in this code.
* Variable A is used in the previous example, so the graphs generated by these examples are now connected. It's probably not a reason for this to fail, but I doubt it's what you intended. Reinitialise A, or use a different variable symbol.
* results is the initial value of the computation (the first value that gets passed to previous). You probably want to set it to 1.0 (depending on how you choose to fix the other issues below).
* updates isn't doing anything right now: there are no shared variables to update.
* The state of the computation (the next value to multiply) is not being passed on.

Of course, this is for educational purposes, and you wouldn't actually compute a factorial this way, so I can't say what is the *correct* way to do it. Here is one way:
* Accumulate the answer as we go along: you're already doing this.
* Use an additional variable to store the next number to multiply with the accumulator.
* Use a conditional ending to terminate the scan.

I can't run this code, and I've never used Theano, so I expect there will be bugs. But hopefully it illustrates the key points.

In [ ]:
def multiplyNext(acc, val):
  return( acc*val, val-1, theano.scan_module.until(val < 2) )

X = T.iscalar("X")

result, updates = theano.scan(
  multiplyNext,
  outputs_info = [ T.constant(1.0), X ],
  strict = True )

f = theano.function( [X], result )

print( f(4) )
print( f(5) )

# Scanning over tensors

In [135]:
import numpy

In [136]:
coefficients = theano.tensor.vector("coefficients")
x = T.scalar("x")

max_coefficients_supported = 10000

# Generate the components of the polynomial
components, updates = theano.scan(fn=lambda coefficient, power, free_variable: coefficient * (free_variable ** power),
                                  outputs_info=None,
                                  sequences=[coefficients, theano.tensor.arange(max_coefficients_supported)],
                                  non_sequences=x)
# Sum them up
polynomial = components.sum()

# Compile a function
calculate_polynomial = theano.function(inputs=[coefficients, x], outputs=polynomial)

# Test
test_coefficients = numpy.asarray([1, 0, 2], dtype=numpy.float32)
test_value = 3
print calculate_polynomial(test_coefficients, test_value)
print 1.0 * (3 ** 0) + 0.0 * (3 ** 1) + 2.0 * (3 ** 2)

19.0
19.0
